In [68]:
import json
import numpy as np
import pandas as pd

import pandas_profiling

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
pd.options.display.max_rows = 2000

# Data loading and cleaning

In [40]:
exports_cafe_df = pd.read_csv( '../data/Cadena_Productiva_Caf__-_Exportaciones.csv', dtype = { 'Anio' : 'str' } )

In [41]:
del exports_cafe_df[ 'Cadena' ]
del exports_cafe_df[ 'CódigoUnidad' ]
del exports_cafe_df[ 'Partida' ]
del exports_cafe_df[ 'ValorMilesPesos' ]
del exports_cafe_df[ 'CantidadUnidades' ]

In [42]:
exports_cafe_df.drop_duplicates( inplace = True )

In [43]:
exports_cafe_df.shape

(46021, 9)

In [44]:
exports_cafe_df.dtypes

Anio                   object
Mes                    object
CódigoPaís              int64
PaísDestino            object
CódigoDepto             int64
DepartamentoOrigen     object
Producto               object
ValorMilesFOBDol      float64
VolúmenToneladas      float64
dtype: object

In [45]:
pandas_profiling.ProfileReport( exports_cafe_df )

Number of variables,10
Number of observations,46021
Total Missing (%),0.0%
Total size in memory,3.5 MiB
Average record size in memory,80.0 B
Numeric,4
Categorical,5
Boolean,0
Date,0
Text (Unique),0
Rejected,1


# Group by Year

In [214]:
grouped_exports_cafe_df = exports_cafe_df.groupby( [ 'DepartamentoOrigen', 'PaísDestino' ] ).agg( { 'VolúmenToneladas' : 'sum' } ).reset_index()

In [215]:
contries_cumsum = exports_cafe_df[ 'PaísDestino' ].value_counts( normalize = True ).cumsum()
less_respres_countries = contries_cumsum.loc[ contries_cumsum > .9 ].index.tolist()
#less_respres_countries

In [216]:
exports_cafe_df = exports_cafe_df[ ~exports_cafe_df[ 'PaísDestino' ].isin( less_respres_countries ) ]

In [217]:
grouped_exports_cafe_df.shape

(563, 3)

In [218]:
grouped_exports_cafe_df.head(10)

,DepartamentoOrigen,PaísDestino,VolúmenToneladas
0,Amazonas,Estados Unidos,76.85
1,Antioquia,Alemania,71372.89
2,Antioquia,Argentina,624.96
3,Antioquia,Aruba,280.40
4,Antioquia,Australia,4612.34
5,Antioquia,Bolivia,240.48
6,Antioquia,Bélgica,44767.37
7,Antioquia,Canadá,76838.78
8,Antioquia,Chile,2519.28
9,Antioquia,China,2653.31


# Network Generator

### Nodes

In [219]:
origenes = list( grouped_exports_cafe_df[ 'DepartamentoOrigen' ].unique() )
destinos = list( grouped_exports_cafe_df[ 'PaísDestino' ].unique() )

In [220]:
nodes = []

for origen in origenes:
    vulumenTotal = grouped_exports_cafe_df.loc[ grouped_exports_cafe_df[ 'DepartamentoOrigen' ] == origen, 'VolúmenToneladas' ].sum()
    vulumenTotal_norm = ( grouped_exports_cafe_df.loc[ grouped_exports_cafe_df[ 'DepartamentoOrigen' ] == origen, 'VolúmenToneladas' ].sum() / grouped_exports_cafe_df[ 'VolúmenToneladas' ].sum() )
    nodes.append( { 'id' : origen, 
                   'group' : 1, 
                   'volumenTotal' : vulumenTotal,
                   'volumenTotal_norm' : vulumenTotal_norm } )

for destino in destinos:
    vulumenTotal = grouped_exports_cafe_df.loc[ grouped_exports_cafe_df[ 'PaísDestino' ] == destino, 'VolúmenToneladas' ].sum()
    vulumenTotal_norm = ( grouped_exports_cafe_df.loc[ grouped_exports_cafe_df[ 'PaísDestino' ] == destino, 'VolúmenToneladas' ].sum() / grouped_exports_cafe_df[ 'VolúmenToneladas' ].sum() )
    nodes.append( { 'id' : destino, 
                   'group' : 2,
                   'volumenTotal' : vulumenTotal,
                   'volumenTotal_norm' : vulumenTotal_norm } )

In [221]:
len( nodes )

74

### Links

In [222]:
links = []

for i, row in grouped_exports_cafe_df.iterrows():
    links.append( { 'source' : row[ 'DepartamentoOrigen' ], 'target' : row[ 'PaísDestino' ], 'value' : row[ 'VolúmenToneladas' ] } )

In [223]:
len( links )

563

In [224]:
data = dict()
data[ 'nodes' ] = nodes
data[ 'links' ] = links

In [225]:
with open( '../data/exports_cafe.json', 'w' ) as outfile:
    json.dump( data, outfile )